<a href="https://colab.research.google.com/github/MuhammadHamzaAfzal/Python_Class_Work_jz/blob/master/Sonar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Baseline Neural Network Model Performance


In [0]:
import numpy
import pandas as pd
from tensorflow.keras import models
from keras.models import Sequential
from tensorflow.keras import layers
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [0]:
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

In [0]:
# load dataset
dataframe = pandas.read_csv("sonar.csv", header=None)
dataset = dataframe.values
# split into input (X) and output (Y) variables
X = dataset[:,0:60].astype(float)
Y = dataset[:,60]


In [19]:
from sklearn.preprocessing import LabelEncoder
label_encoder_op = LabelEncoder()
encoded_Y=label_encoder_op.fit_transform(Y)
encoded_Y

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [20]:
X.shape

(208, 60)

In [0]:
encoded_Y=label_encoder_op.fit_transform(Y)

In [0]:
# baseline model
def create_baseline():
	# create model, write code below
  model=models.Sequential()
  model.add(layers.Dense(60,activation = 'relu',input_shape=(60,)))
  model.add(layers.Dense(1, activation='sigmoid'))
  # Compile model, write code below
  model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
  return model


In [24]:
# evaluate model with standardized dataset
estimator = KerasClassifier(build_fn=create_baseline, epochs=100, batch_size=5, verbose=0)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(estimator, X, encoded_Y, cv=kfold)
print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Results: 84.14% (5.67%)


# Baseline Model With Data Preparation (Pipeline)

In [25]:
# evaluate baseline model with standardized dataset
numpy.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_baseline, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Standardized: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Standardized: 84.14% (6.77%)


# Evaluate a Smaller Network

In [26]:
# smaller model
def create_smaller():
	# create model
  model=models.Sequential()
  model.add(layers.Dense(30,activation = 'relu',input_shape=(60,)))
  model.add(layers.Dense(1, activation='sigmoid'))
	# Compile model
  model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
  return model
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_smaller, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Smaller: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


Smaller: 85.09% (5.02%)


# Evaluate a Larger Network

In [28]:
# larger model
def create_larger():
	# create model
  model=models.Sequential()
  model.add(layers.Dense(60,activation = 'relu',input_shape=(60,)))
  model.add(layers.Dense(30, activation='relu'))
  model.add(layers.Dense(1, activation='sigmoid'))
	# Compile model
  model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
  return model
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_larger, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Larger: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


Larger: 86.06% (3.93%)


# Scaling up: Developing a model that overfits

In [29]:
def create_overfit():
	# create model
  model=models.Sequential()
  model.add(layers.Dense(200,activation = 'relu',input_shape=(60,)))
  model.add(layers.Dense(160, activation='relu'))
  model.add(layers.Dense(130, activation='relu'))
  model.add(layers.Dense(50, activation='relu'))
  model.add(layers.Dense(1, activation='sigmoid'))
	# Compile model
  model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
  return model
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_overfit, epochs=250, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Overfit: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Overfit: 84.59% (4.23%)


# Tuning the Model

In [30]:
def create_best():
	# create model
  model=models.Sequential()
  model.add(layers.Dense(60,activation = 'relu',input_shape=(60,)))
  model.add(layers.Dense(30, activation='relu'))
  model.add(layers.Dense(1, activation='sigmoid'))
	# Compile model
  model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
  return model
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_best, epochs=100, batch_size=5, verbose=1)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Optimized: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Train on 186 samples
Epoch 1/100
186/186 [==============================] - 1s 4ms/sample - loss: 0.7438 - acc: 0.5538
Epoch 2/100
186/186 [==============================] - 0s 837us/sample - loss: 0.4956 - acc: 0.7903
Epoch 3/100
186/186 [==============================] - 0s 853us/sample - loss: 0.3870 - acc: 0.8495
Epoch 4/100
186/186 [==============================] - 0s 847us/sample - loss: 0.3096 - acc: 0.9032
Epoch 5/100
186/186 [==============================] - 0s 859us/sample - loss: 0.2477 - acc: 0.9301
Epoch 6/100
186/186 [==============================] - 0s 904us/sample - loss: 0.1999 - acc: 0.9462
Epoch 7/100
186/186 [==============================] - 0s 844us/sample - loss: 0.1540 - acc: 0.9731
Epoch 8/100
186/186 [==============================] - 0s 868us/sample - loss: 0.1199 - acc: 0.9731
Epoch 9/100
186/186 [==============================] - 0s 851us/sample - loss: 0.0882 - acc: 0.9946
Epoch 10/100
186/186 [==============================] - 0s 884us/sample - loss: 0

# Rewriting the code without using scikit-learn


In [31]:
encode_Y=pd.get_dummies(Y)
# create model, write code below
def build_model():
  model=models.Sequential()
  model.add(layers.Dense(60,activation = 'relu',input_shape=(60,)))
  model.add(layers.Dense(1, activation='sigmoid'))
# Compile model, write code below
  model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
  return model
k = 10
num_val_samples = len(X) // k
num_epochs = 100
all_scores = []
for i in range(k):
  print('processing fold #', i)
  val_data = X[i * num_val_samples: (i + 1) * num_val_samples]
  val_targets = encoded_Y[i * num_val_samples: (i + 1) * num_val_samples]
  partial_train_data = np.concatenate(
  [X[:i * num_val_samples],
  X[(i + 1) * num_val_samples:]],
  axis=0)
  partial_train_targets = np.concatenate(
  [encoded_Y[:i * num_val_samples],
  encoded_Y[(i + 1) * num_val_samples:]],
  axis=0)
  model = build_model()
  model.fit(partial_train_data, partial_train_targets,
  epochs=num_epochs, batch_size=1, verbose=0)
  val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=0)
  all_scores.append(val_mae)

processing fold # 0
processing fold # 1
processing fold # 2
processing fold # 3
processing fold # 4
processing fold # 5
processing fold # 6
processing fold # 7
processing fold # 8
processing fold # 9


In [32]:
all_scores

[0.45, 0.7, 0.55, 0.9, 0.35, 0.25, 0.7, 0.7, 0.55, 1.0]

In [33]:
np.mean(all_scores)

0.615